In [2]:
import torch
words = open('names.txt', 'r').read().splitlines()

In [3]:
# a dictionary that maps each character to a unique integer
# print(f"{sorted(set(''.join(words)))=}")
chars = sorted(list(set(''.join(words)))) # ['a', 'b', 'c',
itos = dict(enumerate(chars, start=1))
itos[0] = '.'
stoi = {s:i for i,s in itos.items()}
# print(f"{itos=}")

In [4]:
list(words[0])

['e', 'm', 'm', 'a']

In [5]:
block_size = 3

X, Y = [], []
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        Y_add = stoi[ch]
        Y.append(Y_add)
        X.append(context)
        context = context[1:] + [Y_add]
X = torch.tensor(X)
Y = torch.tensor(Y)
print(X, Y)
print(X.shape, Y.shape)

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [26, 26, 25],
        [26, 25, 26],
        [25, 26, 24]]) tensor([ 5, 13, 13,  ..., 26, 24,  0])
torch.Size([228146, 3]) torch.Size([228146])


In [ ]:
# set the parameters
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g) # character embedding (27, 2)
W1 = torch.randn((block_size * 2, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]
print(sum(p.nelement() for p in parameters)) # number of parameters


<generator object <genexpr> at 0x0000014C0A7F82B0>


In [ ]:
emb = C[X] # (N, block_size, 2)
embcat = emb.view(emb.shape[0], -1) # (N, block_size*2)
h = torch.tanh(embcat @ W1 + b1) # (N, 100)
logits = h @ W2 + b2 # (N, 27)
counts = logits.exp()
